In [18]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [19]:
submisson_id_to_hardware = {
        '2025-02-06_06-38-32_e14af1d': '1 * H100 NVL',
        '2025-02-06_17-07-07_ab73cc7': '1 * H100 NVL',
        '2025-02-06_21-32-25_ab73cc7': '4 * H100 NVL',
        '2025-02-04_01-05-29_4c55336': '2 * A100 80GB PCIe',
        '2025-02-06_06-41-29_ea3ed4b': '1 * H100 NVL',
        '2025-02-04_20-52-14_eb664c5': '2 * H100 NVL',
        '2025-02-06_17-06-51_ab73cc7': '1 * H100 NVL',
        '2025-02-06_16-26-53_ab73cc7': '1 * H100 NVL',
        '2025-02-06_08-19-02_ab73cc7': '1 * H100 NVL',
        '2025-02-06_10-44-45_ab73cc7': '1 * H100 NVL',
        '2025-02-06_06-39-35_e14af1d': '1 * H100 NVL',
        '2025-02-06_16-28-26_ab73cc7': '2 * H100 NVL',
        '2025-02-06_17-46-21_ab73cc7': '2 * H100 NVL',
        '2025-02-06_02-10-28_d8c80fe': '1 * RTX 6000',
        '2025-02-06_06-29-56_17d3c65': '1 * RTX 6000',
        '2025-02-06_02-07-38_ea3ed4b': '1 * RTX 6000',
        # '2025-02-06_06-42-07_ea3ed4b': 'A100',
        # '2025-02-06_21-32-25_ab73cc7': 'A100',
        # '2025-02-06_06-40-46_ea3ed4b': 'A100',
        # '2025-02-06_17-07-07_ab73cc7': 'A100',
        # '2025-02-06_02-10-28_d8c80fe': 'A100',
        # '2025-02-06_06-54-21_ab73cc7': 'A100',
        # '2025-02-06_06-29-56_17d3c65': 'A100',
        # '2025-02-06_11-43-18_ab73cc7': 'A100',
        # '2025-02-04_18-20-41_bccb1bd': 'A100',
        # '2025-02-06_06-41-29_ea3ed4b': 'A100',
        # '2025-02-06_06-39-35_e14af1d': 'A100',
        # '2025-02-05_03-44-45_d7c02fb': 'A100',
        # '2025-02-06_08-19-02_ab73cc7': 'A100',
        # '2025-02-05_03-44-51_d7c02fb': 'A100',
        # '2025-02-06_10-44-45_ab73cc7': 'A100',
        # '2025-02-06_06-38-32_e14af1d': 'A100',
        # '2025-02-06_06-38-19_e14af1d': 'A100',
        # '2025-02-04_20-52-14_eb664c5': 'A100',
        # '2025-02-04_01-05-29_4c55336': 'A100',
        # '2025-02-06_16-28-26_ab73cc7': 'A100',
        # '2025-02-06_17-46-21_ab73cc7': 'A100',
        # '2025-02-05_03-18-59_d7c02fb': 'A100',
        # '2025-02-06_17-06-51_ab73cc7': 'A100',
        # '2025-02-06_02-07-38_ea3ed4b': 'A100',
        # '2025-02-06_16-26-53_ab73cc7': 'A100',
        # '2025-02-06_06-40-21_e14af1d': 'A100',
    }

In [20]:
from pathlib import Path
import subprocess
from typing import Any, Dict, List

import fire
import pandas as pd
from scipy.stats import hmean
from hf_bench.benchmark import ResultsTableRow


def get_columns() -> List[str]:
    return ResultsTableRow.__annotations__.keys()


def list_tracked_files(dirpath: str) -> List[str]:
    # Run git ls-tree command and capture output
    cmd = ["git", "ls-tree", "-r", "HEAD", "--name-only", dirpath]
    result = subprocess.run(cmd, capture_output=True, text=True, check=True)
    # Split output into list of files
    files = result.stdout.strip().split("\n")
    # Filter out empty strings
    files = [f for f in files if f]
    return files


def list_staged_files(dirpath: str) -> List[str]:
    cmd = ["git", "diff", "--name-only", "--cached", "HEAD", dirpath]
    result = subprocess.run(cmd, capture_output=True, text=True, check=True)
    files = result.stdout.strip().split("\n")
    files = [f for f in files if f]
    return files


def get_df_concat(dirpath: str) -> pd.DataFrame:
    """
    Get a dataframe of all the results in the given directory.
    """
    filepaths = list_tracked_files(dirpath)
    print(f"Found {len(filepaths)} tracked files in {dirpath}.")
    df_first = pd.read_csv(filepaths[0])
    column_dtypes: Dict[str, Any] = {
        "submission_id": str,
        **{col: df_first[col].dtype for col in get_columns()},
    }
    columns = ["submission_id"] + list(get_columns())
    df = pd.DataFrame(columns=columns).astype(column_dtypes)
    for f in filepaths:
        submission_id: str = Path(f).parent.stem
        df_new = pd.read_csv(f)
        df_new["drafter"] = df_new["drafter"].fillna("No Drafter (Autoregressive)")
        df_new["submission_id"] = submission_id
        df_new = df_new[columns]
        df = pd.concat([df, df_new])
    df["submission_id"] = df["submission_id"].map(lambda x: submisson_id_to_hardware.get(x, f"SUBMISSION_ID_{x}"))
    df.sort_values(
        by=columns,
        inplace=True,
    )
    return df


def get_df_concat_filtered(df_concat: pd.DataFrame, minimum_new_toks: int) -> pd.DataFrame:
    df_concat_filtered = df_concat.set_index(["target", "submission_id", "dataset_path", "dataset_name", "dataset_split"])
    df_low_new_toks = (df_concat_filtered[df_concat_filtered["new_toks"] < minimum_new_toks]
                   .set_index("example_id", append=True)
                   .sort_index())
    index_low_new_toks = df_low_new_toks.index.unique() # Multi-index (target, submission_id, dataset_path, dataset_name, dataset_split, example_id) for which new_toks < 64
    # Remove all the rows corresponding to these multi-indices from df_concat_filtered
    df_concat_filtered.set_index("example_id", inplace=True, append=True)
    df_concat_filtered = df_concat_filtered[~df_concat_filtered.index.isin(index_low_new_toks)]
    df_concat_filtered.reset_index(inplace=True)
    return df_concat_filtered


def get_df_summary_of_results(df_concat: pd.DataFrame) -> pd.DataFrame:
    df_concat.reset_index(drop=True, inplace=True)
    columns_for_index: List[str] = [
        "target",
        "submission_id",
        "dataset_path",
        "drafter",
        "temperature",
    ]
    df_concat.set_index(columns_for_index, inplace=True)
    example_id_nunique = df_concat["example_id"].groupby(columns_for_index).nunique()
    df_summary = example_id_nunique.to_frame()
    df_summary.rename(columns={"example_id": "example_id_nunique"}, inplace=True)
    df_mean_vals = df_concat.groupby(columns_for_index)[["new_toks", "ttft_ms"]].mean()
    df_hmean_vals = df_concat.groupby(columns_for_index)[
        ["tpot_ms", "out_toks_per_sec"]
    ].agg(hmean)
    df_summary = pd.concat([df_summary, df_mean_vals, df_hmean_vals], axis=1)
    # Add the speedups
    df_otps = df_summary[["out_toks_per_sec"]]
    df_otps.reset_index(level="drafter", inplace=True)
    mask_ar = df_otps["drafter"] == "No Drafter (Autoregressive)"
    df_ar_otps = df_otps[mask_ar]
    df_ar_otps.drop(columns=["drafter"], inplace=True)
    # Reset the index of both dataframes to make the division operation simpler
    df_otps_reset = df_otps.reset_index()
    df_ar_otps_reset = df_ar_otps.reset_index()
    # Merge the dataframes on the common index columns
    merge_cols = ["target", "dataset_path", "temperature", "submission_id"]
    df_merged = pd.merge(
        df_otps_reset, df_ar_otps_reset, on=merge_cols, suffixes=("", "_ar")
    )
    # Perform the division
    df_merged["speedup"] = (
        df_merged["out_toks_per_sec"] / df_merged["out_toks_per_sec_ar"]
    )
    # Set back the multi-index structure
    df_speedups = df_merged.set_index(merge_cols + ["drafter"])[["speedup"]]
    df_summary.reset_index(inplace=True)
    df_summary.set_index(
        ["target", "dataset_path", "drafter", "temperature", "submission_id"],
        inplace=True,
    )
    df_summary = df_summary.join(df_speedups)
    # Reorder the multi-index columns
    df_summary.reset_index(inplace=True)
    new_index = ["target", "dataset_path", "submission_id", "temperature", "drafter"]
    df_summary.set_index(new_index, inplace=True)
    df_summary.sort_index(level=new_index, inplace=True)
    return df_summary


def get_df_max_speedup(df_summary: pd.DataFrame) -> pd.DataFrame:
    df_summary.reset_index(inplace=True)
    df_max_speedup = df_summary.loc[
        df_summary.groupby(["target", "dataset_path", "submission_id", "temperature"])[
            "speedup"
        ].idxmax()
    ]
    df_max_speedup.rename(columns={"drafter": "drafter_of_max_speedup"}, inplace=True)
    df_max_speedup.set_index(
        [
            "target",
            "temperature",
            "dataset_path",
            "submission_id",
            "drafter_of_max_speedup",
        ],
        inplace=True,
    )
    df_max_speedup.sort_index(inplace=True)
    return df_max_speedup


dirpath = "/home/projects/dharel/nadavt/repos/hf-bench/benchmark_results"
print("Concatenating all the results CSVs into one dataframe...")
df_concat: pd.DataFrame = get_df_concat(dirpath)
# df_concat.to_csv("results_all.csv", index=False)

minimum_new_toks = 128
print(f"Filtering out experiments with less than {minimum_new_toks} new tokens...")
df_concat_filtered = get_df_concat_filtered(df_concat, minimum_new_toks)

print("Counting the number of unique example IDs for each experiment...")
df_summary: pd.DataFrame = get_df_summary_of_results(df_concat_filtered)
# Round the values to 1 decimal place
df_summary["new_toks"] = df_summary["new_toks"].round(1)
df_summary["ttft_ms"] = df_summary["ttft_ms"].round(1)
df_summary["tpot_ms"] = df_summary["tpot_ms"].round(1)
df_summary["out_toks_per_sec"] = df_summary["out_toks_per_sec"].round(1)
df_summary["speedup"] = df_summary["speedup"].round(2)
# df_summary.to_csv("results_summary.csv", index=True)

print("Getting the maximum speedup for each experiment...")
df_max_speedup: pd.DataFrame = get_df_max_speedup(df_summary.copy())
# df_max_speedup.to_csv("results_max_speedup.csv", index=True)

# df_summary_deepseek_qwen_14b = df_summary[df_summary.index.get_level_values("target").str.startswith("deepseek-ai/DeepSeek-R1-Distill-Qwen-14B")]
# df_summary_deepseek_qwen_14b_temperature_0 = df_summary_deepseek_qwen_14b[df_summary_deepseek_qwen_14b.index.get_level_values("temperature") == 0]

# print(f"Stored both the concatenated dataframe and the summary in {dirpath}.")
# print("Done!")

df_concat.head(5)

Concatenating all the results CSVs into one dataframe...
Found 48 tracked files in /home/projects/dharel/nadavt/repos/hf-bench/benchmark_results.
Filtering out experiments with less than 128 new tokens...
Counting the number of unique example IDs for each experiment...
Getting the maximum speedup for each experiment...


/tmp/ipykernel_54233/3429413622.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ar_otps.drop(columns=["drafter"], inplace=True)


,submission_id,target,dataset_path,dataset_name,dataset_split,num_of_examples,drafter,temperature,example_id,new_toks,ttft_ms,tpot_ms,out_toks_per_sec
0,1 * H100 NVL,deepseek-ai/DeepSeek-R1-Distill-Llama-8B,cnn_dailymail,3.0.0,validation,30,No Drafter (Autoregressive),0,0,512,51.167488,21.863732,45.737845
1,1 * H100 NVL,deepseek-ai/DeepSeek-R1-Distill-Llama-8B,cnn_dailymail,3.0.0,validation,30,No Drafter (Autoregressive),0,1,512,91.585398,21.749902,45.977218
2,1 * H100 NVL,deepseek-ai/DeepSeek-R1-Distill-Llama-8B,cnn_dailymail,3.0.0,validation,30,No Drafter (Autoregressive),0,2,512,41.071653,21.785226,45.902669
3,1 * H100 NVL,deepseek-ai/DeepSeek-R1-Distill-Llama-8B,cnn_dailymail,3.0.0,validation,30,No Drafter (Autoregressive),0,3,512,31.173468,21.819458,45.830653
4,1 * H100 NVL,deepseek-ai/DeepSeek-R1-Distill-Llama-8B,cnn_dailymail,3.0.0,validation,30,No Drafter (Autoregressive),0,4,512,41.478634,21.843728,45.779731


In [21]:
df_concat.describe()

,num_of_examples,temperature,example_id,new_toks,ttft_ms,tpot_ms,out_toks_per_sec
count,10650.0,10650.000000,10650.000000,10650.000000,10650.000000,10650.000000,10650.000000
mean,30.0,0.498592,14.500000,393.104413,429.016698,54.531549,39.551700
std,0.0,0.500021,8.655848,153.698989,553.005558,44.652178,1563.060465
min,30.0,0.000000,0.000000,8.000000,20.835876,0.006199,0.300873
25%,30.0,0.000000,7.000000,254.000000,101.354957,32.555696,14.979037
50%,30.0,0.000000,14.500000,510.000000,212.974191,47.632325,20.994146
75%,30.0,1.000000,22.000000,512.000000,504.370451,66.759968,30.716591
max,30.0,1.000000,29.000000,512.000000,3398.638010,3323.662519,161319.384615


In [22]:
df_concat_filtered.describe()

,example_id,num_of_examples,new_toks,ttft_ms,tpot_ms,out_toks_per_sec
count,8939.000000,8939.0,8939.000000,8939.000000,8939.000000,8939.000000
mean,14.290748,30.0,432.273968,423.353101,53.555507,24.692991
std,8.691367,0.0,119.754933,553.884493,31.208981,13.159275
min,0.000000,30.0,128.000000,20.835876,10.357687,3.702622
25%,7.000000,30.0,374.000000,91.417432,32.009255,15.259817
50%,14.000000,30.0,512.000000,203.398943,47.585703,21.014715
75%,22.000000,30.0,512.000000,478.026986,65.531587,31.240965
max,29.000000,30.0,512.000000,3398.638010,270.078875,96.546653


In [23]:
df_summary.head(5)

example_id_nunique  \
target                                    dataset_path  submission_id temperature drafter                                                         
deepseek-ai/DeepSeek-R1-Distill-Llama-70B cnn_dailymail 2 * H100 NVL  0           No Drafter (Autoregressive)                                29   
                                                                                  deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   29   
                                                                                  deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B                  29   
                                                                                  double7/vicuna-68m                                         29   
                                                                                  meta-llama/Llama-3.1-8B                                    29   

                                                                                                                             new_toks  \
target                                    dataset_path  submission_id temperature drafter                                               
deepseek-ai/DeepSeek-R1-Distill-Llama-70B cnn_dailymail 2 * H100 NVL  0           No Drafter (Autoregressive)                   494.1   
                                                                                  deepseek-ai/DeepSeek-R1-Distill-Llama-8B      492.0   
                                                                                  deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B     496.0   
                                                                                  double7/vicuna-68m                            496.0   
                                                                                  meta-llama/Llama-3.1-8B                       494.8   

                                                                                                                             ttft_ms  \
target                                    dataset_path  submission_id temperature drafter                                              
deepseek-ai/DeepSeek-R1-Distill-Llama-70B cnn_dailymail 2 * H100 NVL  0           No Drafter (Autoregressive)                  230.7   
                                                                                  deepseek-ai/DeepSeek-R1-Distill-Llama-8B     452.5   
                                                                                  deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B    296.1   
                                                                                  double7/vicuna-68m                           263.8   
                                                                                  meta-llama/Llama-3.1-8B                      277.9   

                                                                                                                             tpot_ms  \
target                                    dataset_path  submission_id temperature drafter                                              
deepseek-ai/DeepSeek-R1-Distill-Llama-70B cnn_dailymail 2 * H100 NVL  0           No Drafter (Autoregressive)                   77.6   
                                                                                  deepseek-ai/DeepSeek-R1-Distill-Llama-8B      78.7   
                                                                                  deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B     72.3   
                                                                                  double7/vicuna-68m                            51.5   
                                                                                  meta-llama/Llama-3.1-8B                       74.4   

                                                                                                                             out_toks_per_sec  \
target                                    dataset_path  submission_id temperature dr

In [24]:
df_max_speedup.head(5)

example_id_nunique  \
target                                    temperature dataset_path            submission_id      drafter_of_max_speedup                                          
deepseek-ai/DeepSeek-R1-Distill-Llama-70B 0           cnn_dailymail           2 * H100 NVL       meta-llama/Llama-3.2-1B                                    29   
                                                      openai/openai_humaneval 2 * A100 80GB PCIe deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   30   
                                                                              2 * H100 NVL       deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   30   
                                                      tau/scrolls             2 * H100 NVL       deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B                  30   
                                          1           cnn_dailymail           2 * H100 NVL       meta-llama/Llama-3.2-1B                                    29   

                                                                                                                                            new_toks  \
target                                    temperature dataset_path            submission_id      drafter_of_max_speedup                                
deepseek-ai/DeepSeek-R1-Distill-Llama-70B 0           cnn_dailymail           2 * H100 NVL       meta-llama/Llama-3.2-1B                       493.1   
                                                      openai/openai_humaneval 2 * A100 80GB PCIe deepseek-ai/DeepSeek-R1-Distill-Llama-8B      512.0   
                                                                              2 * H100 NVL       deepseek-ai/DeepSeek-R1-Distill-Llama-8B      512.0   
                                                      tau/scrolls             2 * H100 NVL       deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B     464.4   
                                          1           cnn_dailymail           2 * H100 NVL       meta-llama/Llama-3.2-1B                       479.0   

                                                                                                                                            ttft_ms  \
target                                    temperature dataset_path            submission_id      drafter_of_max_speedup                               
deepseek-ai/DeepSeek-R1-Distill-Llama-70B 0           cnn_dailymail           2 * H100 NVL       meta-llama/Llama-3.2-1B                      242.3   
                                                      openai/openai_humaneval 2 * A100 80GB PCIe deepseek-ai/DeepSeek-R1-Distill-Llama-8B     353.5   
                                                                              2 * H100 NVL       deepseek-ai/DeepSeek-R1-Distill-Llama-8B     223.5   
                                                      tau/scrolls             2 * H100 NVL       deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B   1890.9   
                                          1           cnn_dailymail           2 * H100 NVL       meta-llama/Llama-3.2-1B                      248.5   

                                                                                                                                            tpot_ms  \
target                                    temperature dataset_path            submission_id      drafter_of_max_speedup                               
deepseek-ai/DeepSeek-R1-Distill-Llama-70B 0           cnn_dailymail           2 * H100 NVL       meta-llama/Llama-3.2-1B                       51.4   
                                                      openai/openai_humaneval 2 * A100 80GB PCIe deepseek-ai/DeepSeek-R1-Distill-Llama-8B      54.3   
                                                                              2 * H100 NVL       deepseek-ai/DeepSeek-R1-Distill-Llama-8B      52.8   
                                                      tau/scrolls             2 * H100 NVL       deepseek-ai/DeepSeek-R1-Distil

In [25]:
mask_deepseek_target = df_max_speedup.index.get_level_values("target").str.startswith("deepseek")
df_deepseek = df_max_speedup[mask_deepseek_target]
df_deepseek

example_id_nunique  \
target                                    temperature dataset_path            submission_id                             drafter_of_max_speedup                                          
deepseek-ai/DeepSeek-R1-Distill-Llama-70B 0           cnn_dailymail           2 * H100 NVL                              meta-llama/Llama-3.2-1B                                    29   
                                                      openai/openai_humaneval 2 * A100 80GB PCIe                        deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   30   
                                                                              2 * H100 NVL                              deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   30   
                                                      tau/scrolls             2 * H100 NVL                              deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B                  30   
                                          1           cnn_dailymail           2 * H100 NVL                              meta-llama/Llama-3.2-1B                                    29   
                                                      openai/openai_humaneval 2 * A100 80GB PCIe                        deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   30   
                                                                              2 * H100 NVL                              deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   30   
                                                      tau/scrolls             2 * H100 NVL                              deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   30   
deepseek-ai/DeepSeek-R1-Distill-Llama-8B  0           cnn_dailymail           1 * H100 NVL                              double7/vicuna-68m                                         30   
                                                                              SUBMISSION_ID_2025-02-05_03-18-59_d7c02fb double7/vicuna-68m                                         30   
                                                      openai/openai_humaneval 1 * H100 NVL                              No Drafter (Autoregressive)                                30   
                                                                              1 * RTX 6000                              No Drafter (Autoregressive)                                30   
                                                      tau/scrolls             1 * H100 NVL                              meta-llama/Llama-3.2-1B                                    29   
                                                                              SUBMISSION_ID_2025-02-05_03-18-59_d7c02fb meta-llama/Llama-3.2-1B                                    30   
                                          1           cnn_dailymail           1 * H100 NVL                              double7/vicuna-68m                                         30   
                                                                              SUBMISSION_ID_2025-02-05_03-18-59_d7c02fb double7/vicuna-68m                                         30   
                                                      openai/openai_humaneval 1 * H100 NVL                              No Drafter (Autoregressive)                                30   
                                                                              1 * RTX 6000                              No Drafter (Autoregressive)                                30   
                                                      tau/scrolls             1 * H100 NVL                              meta-llama/Llama-3.2-1B                                    29   
                                                                              SUBMISSION_ID_2025-02-05_03-18-59_d7c02fb meta-llama/Llama-3.2-1B                                    30   
deepseek-ai/DeepSeek-R1-Distill-Qwen-14B  0           cnn_dailymail           1 * RTX 6000   

In [26]:
def get_df_summary_for_latex_strings(df: pd.DataFrame, columns_to_drop: List[str] = []) -> pd.DataFrame:
    df = df.reset_index()
    df.drop(columns=columns_to_drop, inplace=True)
    # Split the columns by "/" and take the last element
    columns_to_split = ["dataset_path", "drafter", "target"]
    for col in columns_to_split:
        if col in df.columns:
            df[col] = df[col].str.split("/").str[-1]
    columns_to_rename = {
        "submission_id": "Hardware", "dataset_path": "Dataset", "ttft_ms": "TTFT (ms)", "tpot_ms": "TPOT (ms)", "out_toks_per_sec": "T/s",
         **{col: col.capitalize() for col in ["target", "temperature", "method", "drafter", "speedup"]}
    }
    df.rename(columns=columns_to_rename, inplace=True)
    columns_to_drop_rename = [columns_to_rename.get(col, col) for col in columns_to_drop]
    cols_to_index = [col for col in ["Temperature", "Target", "Dataset", "Hardware", "Method", "Drafter"] if col in df.columns and col not in columns_to_drop_rename]
    df.set_index(cols_to_index, inplace=True)
    return df

columns_to_drop = ["target", "temperature", "example_id_nunique", "new_toks"]
# df_summary_deepseek_qwen_14b_temperature_0 = get_df_summary_for_latex_strings(df_summary_deepseek_qwen_14b_temperature_0.copy(), columns_to_drop)
# df_summary_deepseek_qwen_14b_temperature_0

In [27]:
# # def get_df_summary_sort_by_speedup(df_summary: pd.DataFrame, columns_to_group_by: List[str]) -> pd.DataFrame:
# #     # Get maximum speedup for each combination of grouping columns
# #     max_speedups = df_summary.groupby(columns_to_group_by)['Speedup'].max()
    
# #     # Sort hierarchically from outer to inner columns
# #     sorted_index = []
    
# #     def sort_level(current_index: tuple, level: int = 0):
# #         if level == len(columns_to_group_by):
# #             # Get all rows matching the current grouping columns
# #             mask = pd.Series(True, index=df_summary.index)
# #             for i, val in enumerate(current_index):
# #                 mask &= df_summary.index.get_level_values(i) == val
# #             sorted_index.extend(df_summary[mask].index)
# #             return
            
# #         # Get all values at current level that match the current_index
# #         if level == 0:
# #             current_max_speedups = max_speedups
# #         else:
# #             # Create a boolean mask for all previous levels
# #             mask = pd.Series(True, index=max_speedups.index)
# #             for i, val in enumerate(current_index):
# #                 mask &= max_speedups.index.get_level_values(i) == val
# #             current_max_speedups = max_speedups[mask]
            
# #         # Get unique values at current level and sort by their maximum speedups
# #         current_level_values = current_max_speedups.index.get_level_values(level).unique()
# #         level_speedups = {val: current_max_speedups[current_max_speedups.index.get_level_values(level) == val].max() 
# #                          for val in current_level_values}
# #         sorted_level_values = sorted(level_speedups.keys(), key=lambda x: level_speedups[x], reverse=True)
        
# #         # Recurse to next level for each sorted value
# #         for value in sorted_level_values:
# #             new_index = current_index + (value,) if level > 0 else (value,)
# #             sort_level(new_index, level + 1)
    
# #     # Start recursive sorting from the outermost level
# #     sort_level(())
    
# #     # Reindex the dataframe with the sorted index
# #     df_summary = df_summary.reindex(dict.fromkeys(sorted_index))  # dict.fromkeys preserves order while removing duplicates
# #     return df_summary


# def get_df_summary_sort_by_speedup(df_summary: pd.DataFrame, columns_to_group_by: List[str]) -> pd.DataFrame:
#     # Get maximum speedup for each combination of grouping columns
#     max_speedups = df_summary.groupby(columns_to_group_by)['Speedup'].max()
    
#     def get_method_priority(method: str) -> int:
#         # Higher priority = sorted first
#         if any(m in method for m in ["SLEM", "TLI"]):
#             return 2
#         elif "SD" in method:
#             return 1
#         elif "AR" in method:
#             return 0
#         return -1  # Any other method gets lowest priority
    
#     def get_group_priority_and_speedup(group_index):
#         # Get all rows for this group
#         mask = pd.Series(True, index=df_summary.index)
#         for i, val in enumerate(group_index):
#             mask &= df_summary.index.get_level_values(i) == val
#         group_data = df_summary[mask]
        
#         # Find the row with maximum speedup
#         max_speedup_idx = group_data['Speedup'].idxmax()
#         max_speedup = group_data.loc[max_speedup_idx, 'Speedup']
        
#         # Get the method for this maximum speedup
#         method = ""
#         if "Method" in df_summary.index.names:
#             method = max_speedup_idx[df_summary.index.names.index("Method")]
#         elif "Method" in df_summary.columns:
#             method = group_data.loc[max_speedup_idx, "Method"]
#         return (get_method_priority(method), max_speedup)
    
#     # Sort hierarchically from outer to inner columns
#     sorted_index = []
    
#     def sort_level(current_index: tuple, level: int = 0):
#         if level == len(columns_to_group_by):
#             # Get all rows matching the current grouping columns
#             mask = pd.Series(True, index=df_summary.index)
#             for i, val in enumerate(current_index):
#                 mask &= df_summary.index.get_level_values(i) == val
#             sorted_index.extend(df_summary[mask].index)
#             return
            
#         # Get all values at current level that match the current_index
#         if level == 0:
#             current_max_speedups = max_speedups
#         else:
#             mask = pd.Series(True, index=max_speedups.index)
#             for i, val in enumerate(current_index):
#                 mask &= max_speedups.index.get_level_values(i) == val
#             current_max_speedups = max_speedups[mask]
            
#         # Get unique values at current level and sort by method priority and speedup
#         current_level_values = current_max_speedups.index.get_level_values(level).unique()
        
#         # Create sorting key that considers both method priority and speedup
#         level_values_with_priority = {
#             val: get_group_priority_and_speedup(current_index + (val,) if level > 0 else (val,))
#             for val in current_level_values
#         }
        
#         # Sort first by method priority (descending), then by speedup (descending)
#         sorted_level_values = sorted(
#             current_level_values,
#             key=lambda x: level_values_with_priority[x],
#             reverse=True
#         )
        
#         # Recurse to next level for each sorted value
#         for value in sorted_level_values:
#             new_index = current_index + (value,) if level > 0 else (value,)
#             sort_level(new_index, level + 1)
    
#     # Start recursive sorting from the outermost level
#     sort_level(())
    
#     # Reindex the dataframe with the sorted index
#     df_summary = df_summary.reindex(dict.fromkeys(sorted_index))
#     return df_summary

# df_summary_deepseek_qwen_14b_temperature_0 = get_df_summary_sort_by_speedup(df_summary_deepseek_qwen_14b_temperature_0, columns_to_group_by=["Dataset", "Hardware"])
# df_summary_deepseek_qwen_14b_temperature_0

In [28]:
def hierarchical_max_sort(df: pd.DataFrame, 
                         value_col: str,
                         index_levels: List[str] = None) -> pd.DataFrame:
    """
    Sort a DataFrame hierarchically by maximum values at each level of the index.
    
    Args:
        df: DataFrame with multi-index
        value_col: Name of the column to sort by
        index_levels: List of index level names. If None, uses all levels in order
    
    Returns:
        Sorted DataFrame
    """
    if index_levels is None:
        index_levels = list(df.index.names)
    
    df_temp = df.copy()
    
    # Add maximum value columns for each level
    for i in range(len(index_levels)):
        level_name = f'level{i+1}_max'
        group_levels = index_levels[:i+1]
        df_temp[level_name] = df.groupby(group_levels)[value_col].transform('max')
    
    # Sort by all max columns
    max_cols = [f'level{i+1}_max' for i in range(len(index_levels))]
    df_sorted = df_temp.sort_values(
        max_cols,
        ascending=[False] * len(max_cols)
    )[value_col].to_frame()
    
    return df_sorted

In [29]:
def get_df_summary_for_latex_floats(df_summary: pd.DataFrame, columns_to_bold_max: List[str]) -> pd.DataFrame:
    """
    For each column in the dataframe, find the index of the maximum or minimum value.
    If the column is in the list of columns to bold, bold the maximum value, otherwise bold the minimum value.
    """
    df_summary["Speedup"] = df_summary["Speedup"].round(2)
    cols_to_groupby: List[str] = df_summary.index.names.difference(["Method", "Drafter"])
    for col in df_summary.columns:
        g = df_summary.groupby(cols_to_groupby)[col]
        index_to_bold = g.idxmax() if col in columns_to_bold_max else g.idxmin()
        df_summary[col] = df_summary[col].astype(str)
        df_summary.loc[index_to_bold, col] = df_summary.loc[index_to_bold, col].apply(lambda x: "\\textbf{" + x + "}")
    return df_summary

# For each multi-index (Dataset, Hardware), find the maximum Speedup
columns_to_bold_max = ["T/s", "Speedup"]
# df_summary_deepseek_qwen_14b_temperature_0 = get_df_summary_for_latex_floats(df_summary_deepseek_qwen_14b_temperature_0, columns_to_bold_max)
# df_summary_deepseek_qwen_14b_temperature_0

In [30]:
import re


def get_latex_table(df: pd.DataFrame, longtable: bool) -> str:
    table: str = df.to_latex(escape=True, longtable=longtable)
    table = re.sub(r'\\textbackslash textbf\\\{([\d.]+)\\\}', r'\\textbf{\1}', table)
    return table

# print(get_latex_table(df_summary_deepseek_qwen_14b_temperature_0, longtable=False))

In [31]:
def is_method_sd(target: str, drafter: str) -> str:
    target = target.lower()
    drafter = drafter.lower()
    family = target.split("-")[0]
    if "phi" in family:
        return False
    if "deepseek" in family:
        if "llama" in target and "llama" in drafter:
            return True
        if "qwen" in target and "qwen" in drafter:
            return True
        return False
    if family in drafter:
        return True
    return False

def get_method_name(target: str, drafter: str, temperature: float) -> str:
    if drafter == "No Drafter (Autoregressive)":
        return "AR"
    if is_method_sd(target, drafter):
        return "SD"
    if temperature == 0:
        return "SLEM"
    return "TLI"


def enrich_df_summary_with_method_name(df_summary: pd.DataFrame) -> pd.DataFrame:
    # We will insert the "Method" column between `left_index_cols` and `right_index_cols`
    cols_index_right = ["drafter"]
    cols_index_left = df_summary.index.names.difference(cols_index_right)
    df_summary.reset_index(inplace=True)
    df_summary["method"] = df_summary.apply(lambda x: get_method_name(x["target"], x["drafter"], x["temperature"]), axis=1)
    df_summary.set_index(cols_index_left + ["method"] + cols_index_right, inplace=True)
    return df_summary


df_summary = enrich_df_summary_with_method_name(df_summary)
df_summary.head(10)

example_id_nunique  \
target                                    dataset_path  submission_id temperature method drafter                                                         
deepseek-ai/DeepSeek-R1-Distill-Llama-70B cnn_dailymail 2 * H100 NVL  0           AR     No Drafter (Autoregressive)                                29   
                                                                                  SD     deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   29   
                                                                                  SLEM   deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B                  29   
                                                                                         double7/vicuna-68m                                         29   
                                                                                  SD     meta-llama/Llama-3.1-8B                                    29   
                                                                                         meta-llama/Llama-3.2-1B                                    29   
                                                                                         meta-llama/Llama-3.2-3B                                    29   
                                                                      1           AR     No Drafter (Autoregressive)                                29   
                                                                                  SD     deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   29   
                                                                                  TLI    deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B                  29   

                                                                                                                                    new_toks  \
target                                    dataset_path  submission_id temperature method drafter                                               
deepseek-ai/DeepSeek-R1-Distill-Llama-70B cnn_dailymail 2 * H100 NVL  0           AR     No Drafter (Autoregressive)                   494.1   
                                                                                  SD     deepseek-ai/DeepSeek-R1-Distill-Llama-8B      492.0   
                                                                                  SLEM   deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B     496.0   
                                                                                         double7/vicuna-68m                            496.0   
                                                                                  SD     meta-llama/Llama-3.1-8B                       494.8   
                                                                                         meta-llama/Llama-3.2-1B                       493.1   
                                                                                         meta-llama/Llama-3.2-3B                       492.4   
                                                                      1           AR     No Drafter (Autoregressive)                   480.6   
                                                                                  SD     deepseek-ai/DeepSeek-R1-Distill-Llama-8B      458.3   
                                                                                  TLI    deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B     500.2   

                                                                                                                                    ttft_ms  \
target                                    dataset_path  submission_id temperature method drafter                                              
deepseek-ai/DeepSeek-R1-Distill-Llama-70B cnn_dailymail 2 * H100 NVL  0           AR     No Drafter (Autoregressive)                  230.7   
                                                                                  SD     deepseek-ai/DeepSeek-R1-Distill-Llam

In [32]:
df_summary_for_latex = get_df_summary_for_latex_strings(df_summary, columns_to_drop=["example_id_nunique", "new_toks"])
df_summary_for_latex.head()

TTFT (ms)  \
Temperature Target                        Dataset       Hardware     Method Drafter                                    
0           DeepSeek-R1-Distill-Llama-70B cnn_dailymail 2 * H100 NVL AR     No Drafter (Autoregressive)        230.7   
                                                                     SD     DeepSeek-R1-Distill-Llama-8B       452.5   
                                                                     SLEM   DeepSeek-R1-Distill-Qwen-1.5B      296.1   
                                                                            vicuna-68m                         263.8   
                                                                     SD     Llama-3.1-8B                       277.9   

                                                                                                           TPOT (ms)  \
Temperature Target                        Dataset       Hardware     Method Drafter                                    
0           DeepSeek-R1-Distill-Llama-70B cnn_dailymail 2 * H100 NVL AR     No Drafter (Autoregressive)         77.6   
                                                                     SD     DeepSeek-R1-Distill-Llama-8B        78.7   
                                                                     SLEM   DeepSeek-R1-Distill-Qwen-1.5B       72.3   
                                                                            vicuna-68m                          51.5   
                                                                     SD     Llama-3.1-8B                        74.4   

                                                                                                            T/s  \
Temperature Target                        Dataset       Hardware     Method Drafter                               
0           DeepSeek-R1-Distill-Llama-70B cnn_dailymail 2 * H100 NVL AR     No Drafter (Autoregressive)    12.9   
                                                                     SD     DeepSeek-R1-Distill-Llama-8B   12.5   
                                                                     SLEM   DeepSeek-R1-Distill-Qwen-1.5B  13.6   
                                                                            vicuna-68m                     19.3   
                                                                     SD     Llama-3.1-8B                   13.4   

                                                                                                           Speedup  
Temperature Target                        Dataset       Hardware     Method Drafter                                 
0           DeepSeek-R1-Distill-Llama-70B cnn_dailymail 2 * H100 NVL AR     No Drafter (Autoregressive)       1.00  
                                                                     SD     DeepSeek-R1-Distill-Llama-8B      0.97  
                                                                     SLEM   DeepSeek-R1-Distill-Qwen-1.5B     1.06  
                                                                            vicuna-68m                        1.50  
                                                                     SD     Llama-3.1-8B                      1.04

In [33]:
def get_df_filtered_out_of_unknown_hardware(df: pd.DataFrame) -> pd.DataFrame:
    return df[~df.index.get_level_values("Hardware").str.startswith("SUBMISSION_ID_")]


df_summary_for_latex_with_hardware = get_df_filtered_out_of_unknown_hardware(df_summary_for_latex)
df_summary_for_latex_with_hardware.head()

TTFT (ms)  \
Temperature Target                        Dataset       Hardware     Method Drafter                                    
0           DeepSeek-R1-Distill-Llama-70B cnn_dailymail 2 * H100 NVL AR     No Drafter (Autoregressive)        230.7   
                                                                     SD     DeepSeek-R1-Distill-Llama-8B       452.5   
                                                                     SLEM   DeepSeek-R1-Distill-Qwen-1.5B      296.1   
                                                                            vicuna-68m                         263.8   
                                                                     SD     Llama-3.1-8B                       277.9   

                                                                                                           TPOT (ms)  \
Temperature Target                        Dataset       Hardware     Method Drafter                                    
0           DeepSeek-R1-Distill-Llama-70B cnn_dailymail 2 * H100 NVL AR     No Drafter (Autoregressive)         77.6   
                                                                     SD     DeepSeek-R1-Distill-Llama-8B        78.7   
                                                                     SLEM   DeepSeek-R1-Distill-Qwen-1.5B       72.3   
                                                                            vicuna-68m                          51.5   
                                                                     SD     Llama-3.1-8B                        74.4   

                                                                                                            T/s  \
Temperature Target                        Dataset       Hardware     Method Drafter                               
0           DeepSeek-R1-Distill-Llama-70B cnn_dailymail 2 * H100 NVL AR     No Drafter (Autoregressive)    12.9   
                                                                     SD     DeepSeek-R1-Distill-Llama-8B   12.5   
                                                                     SLEM   DeepSeek-R1-Distill-Qwen-1.5B  13.6   
                                                                            vicuna-68m                     19.3   
                                                                     SD     Llama-3.1-8B                   13.4   

                                                                                                           Speedup  
Temperature Target                        Dataset       Hardware     Method Drafter                                 
0           DeepSeek-R1-Distill-Llama-70B cnn_dailymail 2 * H100 NVL AR     No Drafter (Autoregressive)       1.00  
                                                                     SD     DeepSeek-R1-Distill-Llama-8B      0.97  
                                                                     SLEM   DeepSeek-R1-Distill-Qwen-1.5B     1.06  
                                                                            vicuna-68m                        1.50  
                                                                     SD     Llama-3.1-8B                      1.04

In [34]:
df_summary_for_latex_with_hardware_sorted = hierarchical_max_sort(df_summary_for_latex_with_hardware, value_col="Speedup", index_levels=["Temperature", "Target", "Dataset", "Hardware"])
df_summary_for_latex_with_hardware_sorted.head()

Speedup
Temperature Target        Dataset          Hardware     Method Drafter                             
0           gemma-2-9b-it scrolls          1 * H100 NVL AR     No Drafter (Autoregressive)     1.00
                                                        SLEM   vicuna-68m                      1.87
                                                        SD     gemma-2-2b-it                   3.05
                          openai_humaneval 1 * H100 NVL AR     No Drafter (Autoregressive)     1.00
                                                        SLEM   vicuna-68m                      1.49

In [35]:
df_summary_for_latex_with_hardware_sorted_bolded = get_df_summary_for_latex_floats(df_summary_for_latex_with_hardware_sorted, columns_to_bold_max)
df_summary_for_latex_with_hardware_sorted_bolded

Speedup
Temperature Target                        Dataset          Hardware           Method Drafter                                     
0           gemma-2-9b-it                 scrolls          1 * H100 NVL       AR     No Drafter (Autoregressive)              1.0
                                                                              SLEM   vicuna-68m                              1.87
                                                                              SD     gemma-2-2b-it                  \textbf{3.05}
                                          openai_humaneval 1 * H100 NVL       AR     No Drafter (Autoregressive)              1.0
                                                                              SLEM   vicuna-68m                     \textbf{1.49}
                                                                              SD     gemma-2-2b-it                           1.23
                                          cnn_dailymail    1 * H100 NVL       AR     No Drafter (Autoregressive)              1.0
                                                                              SLEM   vicuna-68m                     \textbf{1.39}
                                                                              SD     gemma-2-2b-it                           0.92
            DeepSeek-R1-Distill-Llama-70B openai_humaneval 2 * A100 80GB PCIe AR     No Drafter (Autoregressive)              1.0
                                                                              SLEM   tiny_starcoder_py                       1.44
                                                                              SD     CodeLlama-7b-Instruct-hf                1.18
                                                                                     DeepSeek-R1-Distill-Llama-8B   \textbf{2.25}
                                                           2 * H100 NVL       AR     No Drafter (Autoregressive)              1.0
                                                                              SLEM   tiny_starcoder_py                       1.19
                                                                              SD     CodeLlama-7b-Instruct-hf                0.79
                                                                                     DeepSeek-R1-Distill-Llama-8B   \textbf{1.43}
                                                                              SLEM   DeepSeek-R1-Distill-Qwen-1.5B           1.24
                                          cnn_dailymail    2 * H100 NVL       AR     No Drafter (Autoregressive)              1.0
                                                                              SD     DeepSeek-R1-Distill-Llama-8B            0.97
                                                                              SLEM   DeepSeek-R1-Distill-Qwen-1.5B           1.06
                                                                                     vicuna-68m                               1.5
                                                                              SD     Llama-3.1-8B                            1.04
                                                                                     Llama-3.2-1B                   \textbf{1.51}
                                                                                     Llama-3.2-3B                            1.16
                                          scrolls          2 * H100 NVL       AR     No Drafter (Autoregressive)              1.0
                                                                              SD     DeepSeek-R1-Distill-Llama-8B            1.42
                                                                              SLEM   DeepSeek-R1-Distill-Qwen-1.5B  \textbf{1.47}
            Mixtral-8x22B-Instruct-v0.1   cnn_dailymail    4 * H100 NVL       AR     No Drafter (Autoregressive)              1.0
                                                                              SLEM   Qwen2.5

In [36]:
# print(get_latex_table(get_df_filtered_out_of_unknown_hardware(df_summary_deepseek_qwen_14b_temperature_0), longtable=False))

In [37]:
def get_df_summary_deepseek_qwen_14b(df_summary: pd.DataFrame, temperature: float) -> pd.DataFrame:
    mask_deepseek_qwen_14b = df_summary.index.get_level_values("Target").str.startswith("DeepSeek-R1-Distill-Qwen-14B")
    mask_temperature = df_summary.index.get_level_values("Temperature") == temperature
    return df_summary[mask_deepseek_qwen_14b & mask_temperature]

df_deepseek_qwen_14b_temperature_1 = get_df_summary_deepseek_qwen_14b(df_summary_for_latex_with_hardware_sorted_bolded, 1)
df_deepseek_qwen_14b_temperature_1

Speedup
Temperature Target                       Dataset          Hardware     Method Drafter                                     
1           DeepSeek-R1-Distill-Qwen-14B scrolls          1 * RTX 6000 AR     No Drafter (Autoregressive)              1.0
                                                                       SD     DeepSeek-R1-Distill-Qwen-1.5B            1.5
                                                                       TLI    vicuna-68m                     \textbf{1.59}
                                         cnn_dailymail    1 * RTX 6000 AR     No Drafter (Autoregressive)              1.0
                                                                       SD     DeepSeek-R1-Distill-Qwen-1.5B           1.16
                                                                       TLI    vicuna-68m                     \textbf{1.31}
                                         openai_humaneval 1 * RTX 6000 AR     No Drafter (Autoregressive)              1.0
                                                                       TLI    tiny_starcoder_py               \textbf{1.3}
                                                                              CodeLlama-7b-Instruct-hf                0.75
                                                                       SD     DeepSeek-R1-Distill-Qwen-1.5B           1.08

In [38]:
def drop_multiindex_columns(df: pd.DataFrame, multiindex_columns_to_drop: List[str]) -> pd.DataFrame:
    df = df.copy()
    index_cols = df.index.names
    df.reset_index(drop=False, inplace=True)
    df.drop(multiindex_columns_to_drop, axis=1, inplace=True)
    new_index_cols = index_cols.difference(multiindex_columns_to_drop)
    df.set_index(new_index_cols, inplace=True)
    return df

df_deepseek_qwen_14b_temperature_1 = drop_multiindex_columns(df_deepseek_qwen_14b_temperature_1, ["Temperature", "Target"])
df_deepseek_qwen_14b_temperature_1

Speedup
Dataset          Hardware     Method Drafter                                     
scrolls          1 * RTX 6000 AR     No Drafter (Autoregressive)              1.0
                              SD     DeepSeek-R1-Distill-Qwen-1.5B            1.5
                              TLI    vicuna-68m                     \textbf{1.59}
cnn_dailymail    1 * RTX 6000 AR     No Drafter (Autoregressive)              1.0
                              SD     DeepSeek-R1-Distill-Qwen-1.5B           1.16
                              TLI    vicuna-68m                     \textbf{1.31}
openai_humaneval 1 * RTX 6000 AR     No Drafter (Autoregressive)              1.0
                              TLI    tiny_starcoder_py               \textbf{1.3}
                                     CodeLlama-7b-Instruct-hf                0.75
                              SD     DeepSeek-R1-Distill-Qwen-1.5B           1.08

In [39]:
print(get_latex_table(get_df_filtered_out_of_unknown_hardware(df_deepseek_qwen_14b_temperature_1), longtable=False))

\begin{tabular}{lllll}
\toprule
 &  &  &  & Speedup \\
Dataset & Hardware & Method & Drafter &  \\
\midrule
\multirow[t]{3}{*}{scrolls} & \multirow[t]{3}{*}{1 * RTX 6000} & AR & No Drafter (Autoregressive) & 1.0 \\
\cline{3-5}
 &  & SD & DeepSeek-R1-Distill-Qwen-1.5B & 1.5 \\
\cline{3-5}
 &  & TLI & vicuna-68m & \textbf{1.59} \\
\cline{1-5} \cline{2-5} \cline{3-5}
\multirow[t]{3}{*}{cnn\_dailymail} & \multirow[t]{3}{*}{1 * RTX 6000} & AR & No Drafter (Autoregressive) & 1.0 \\
\cline{3-5}
 &  & SD & DeepSeek-R1-Distill-Qwen-1.5B & 1.16 \\
\cline{3-5}
 &  & TLI & vicuna-68m & \textbf{1.31} \\
\cline{1-5} \cline{2-5} \cline{3-5}
\multirow[t]{4}{*}{openai\_humaneval} & \multirow[t]{4}{*}{1 * RTX 6000} & AR & No Drafter (Autoregressive) & 1.0 \\
\cline{3-5}
 &  & \multirow[t]{2}{*}{TLI} & tiny\_starcoder\_py & \textbf{1.3} \\
 &  &  & CodeLlama-7b-Instruct-hf & 0.75 \\
\cline{3-5}
 &  & SD & DeepSeek-R1-Distill-Qwen-1.5B & 1.08 \\
\cline{1-5} \cline{2-5} \cline{3-5}
\bottomrule
\end{tabular

In [40]:
df_deepseek_qwen_14b_temperature_0 = get_df_summary_deepseek_qwen_14b(df_summary_for_latex_with_hardware_sorted_bolded, 0)
df_deepseek_qwen_14b_temperature_0 = drop_multiindex_columns(df_deepseek_qwen_14b_temperature_0, ["Temperature", "Target"])
df_deepseek_qwen_14b_temperature_0

Speedup
Dataset          Hardware     Method Drafter                                     
scrolls          1 * RTX 6000 AR     No Drafter (Autoregressive)              1.0
                              SD     DeepSeek-R1-Distill-Qwen-1.5B           1.48
                              SLEM   vicuna-68m                     \textbf{1.59}
cnn_dailymail    1 * RTX 6000 AR     No Drafter (Autoregressive)              1.0
                              SD     DeepSeek-R1-Distill-Qwen-1.5B           0.73
                              SLEM   vicuna-68m                     \textbf{1.43}
openai_humaneval 1 * RTX 6000 AR     No Drafter (Autoregressive)              1.0
                              SLEM   tiny_starcoder_py              \textbf{1.14}
                                     CodeLlama-7b-Instruct-hf                0.63
                              SD     DeepSeek-R1-Distill-Qwen-1.5B           0.69

In [41]:
print(get_latex_table(get_df_filtered_out_of_unknown_hardware(df_deepseek_qwen_14b_temperature_0), longtable=False))

\begin{tabular}{lllll}
\toprule
 &  &  &  & Speedup \\
Dataset & Hardware & Method & Drafter &  \\
\midrule
\multirow[t]{3}{*}{scrolls} & \multirow[t]{3}{*}{1 * RTX 6000} & AR & No Drafter (Autoregressive) & 1.0 \\
\cline{3-5}
 &  & SD & DeepSeek-R1-Distill-Qwen-1.5B & 1.48 \\
\cline{3-5}
 &  & SLEM & vicuna-68m & \textbf{1.59} \\
\cline{1-5} \cline{2-5} \cline{3-5}
\multirow[t]{3}{*}{cnn\_dailymail} & \multirow[t]{3}{*}{1 * RTX 6000} & AR & No Drafter (Autoregressive) & 1.0 \\
\cline{3-5}
 &  & SD & DeepSeek-R1-Distill-Qwen-1.5B & 0.73 \\
\cline{3-5}
 &  & SLEM & vicuna-68m & \textbf{1.43} \\
\cline{1-5} \cline{2-5} \cline{3-5}
\multirow[t]{4}{*}{openai\_humaneval} & \multirow[t]{4}{*}{1 * RTX 6000} & AR & No Drafter (Autoregressive) & 1.0 \\
\cline{3-5}
 &  & \multirow[t]{2}{*}{SLEM} & tiny\_starcoder\_py & \textbf{1.14} \\
 &  &  & CodeLlama-7b-Instruct-hf & 0.63 \\
\cline{3-5}
 &  & SD & DeepSeek-R1-Distill-Qwen-1.5B & 0.69 \\
\cline{1-5} \cline{2-5} \cline{3-5}
\bottomrule
\end{ta

In [42]:
mask_temperature_0 = df_summary_for_latex_with_hardware_sorted_bolded.index.get_level_values("Temperature") == 0
df_summary_for_latex_sorted_bolded_with_hardware_temperature_0 = drop_multiindex_columns(df_summary_for_latex_with_hardware_sorted_bolded[mask_temperature_0], ["Temperature"])
print(get_latex_table(df_summary_for_latex_sorted_bolded_with_hardware_temperature_0, longtable=True))

\begin{longtable}{llllll}
\toprule
 &  &  &  &  & Speedup \\
Target & Dataset & Hardware & Method & Drafter &  \\
\midrule
\endfirsthead
\toprule
 &  &  &  &  & Speedup \\
Target & Dataset & Hardware & Method & Drafter &  \\
\midrule
\endhead
\midrule
\multicolumn{6}{r}{Continued on next page} \\
\midrule
\endfoot
\bottomrule
\endlastfoot
\multirow[t]{9}{*}{gemma-2-9b-it} & \multirow[t]{3}{*}{scrolls} & \multirow[t]{3}{*}{1 * H100 NVL} & AR & No Drafter (Autoregressive) & 1.0 \\
\cline{4-6}
 &  &  & SLEM & vicuna-68m & 1.87 \\
\cline{4-6}
 &  &  & SD & gemma-2-2b-it & \textbf{3.05} \\
\cline{2-6} \cline{3-6} \cline{4-6}
 & \multirow[t]{3}{*}{openai\_humaneval} & \multirow[t]{3}{*}{1 * H100 NVL} & AR & No Drafter (Autoregressive) & 1.0 \\
\cline{4-6}
 &  &  & SLEM & vicuna-68m & \textbf{1.49} \\
\cline{4-6}
 &  &  & SD & gemma-2-2b-it & 1.23 \\
\cline{2-6} \cline{3-6} \cline{4-6}
 & \multirow[t]{3}{*}{cnn\_dailymail} & \multirow[t]{3}{*}{1 * H100 NVL} & AR & No Drafter (Autoregressive) 

In [43]:
df_summary_for_latex_sorted_bolded_with_hardware_temperature_1 = drop_multiindex_columns(df_summary_for_latex_with_hardware_sorted_bolded[~mask_temperature_0], ["Temperature"])
print(get_latex_table(df_summary_for_latex_sorted_bolded_with_hardware_temperature_1, longtable=True))

\begin{longtable}{llllll}
\toprule
 &  &  &  &  & Speedup \\
Target & Dataset & Hardware & Method & Drafter &  \\
\midrule
\endfirsthead
\toprule
 &  &  &  &  & Speedup \\
Target & Dataset & Hardware & Method & Drafter &  \\
\midrule
\endhead
\midrule
\multicolumn{6}{r}{Continued on next page} \\
\midrule
\endfoot
\bottomrule
\endlastfoot
\multirow[t]{9}{*}{gemma-2-9b-it} & \multirow[t]{3}{*}{scrolls} & \multirow[t]{3}{*}{1 * H100 NVL} & AR & No Drafter (Autoregressive) & 1.0 \\
\cline{4-6}
 &  &  & TLI & vicuna-68m & 2.04 \\
\cline{4-6}
 &  &  & SD & gemma-2-2b-it & \textbf{2.49} \\
\cline{2-6} \cline{3-6} \cline{4-6}
 & \multirow[t]{3}{*}{openai\_humaneval} & \multirow[t]{3}{*}{1 * H100 NVL} & AR & No Drafter (Autoregressive) & 1.0 \\
\cline{4-6}
 &  &  & TLI & vicuna-68m & \textbf{1.46} \\
\cline{4-6}
 &  &  & SD & gemma-2-2b-it & 1.36 \\
\cline{2-6} \cline{3-6} \cline{4-6}
 & \multirow[t]{3}{*}{cnn\_dailymail} & \multirow[t]{3}{*}{1 * H100 NVL} & AR & No Drafter (Autoregressive) & 